In [1]:
import pandas as pd
import numpy as np
import sklearn
import nltk
import re

In [2]:
data = pd.read_csv("hn2018_2019.csv")

In [3]:
def dater(datetim,integer):
    date,time = datetim.split(" ")
    year,month,day = date.split("-")
    hour,minute,second = time.split(":")
    second = int(second)+ int(minute)*60 + int(hour)*60*60
    if integer==1:
        return year
    elif integer==2:
        return month
    elif integer==3:
        return day
    else:
        return second

In [4]:
data["year"] = data['Created At'].map(lambda x: dater(x,1))
data["month"] = data['Created At'].map(lambda x: dater(x,2))
data["day"] = data['Created At'].map(lambda x: dater(x,3))
data["second"] = data['Created At'].map(lambda x: dater(x,4))

In [5]:
df_2018 = data[data["year"]=='2018']

In [6]:
df_2018['tokenized_title'] = df_2018['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x))

/Users/naitikbhise/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [7]:
df_2018['Title'] = df_2018['Title'].map(lambda x:x.lower())

/Users/naitikbhise/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [8]:
df_2018['tokenized_title'] = df_2018['Title'].map(lambda x:re.split('\[\^a-zA-Z\]',x)[0].split())

/Users/naitikbhise/miniconda3/lib/python3.5/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [9]:
df_2018.head()

,Unnamed: 0,Object ID,Title,Post Type,Author,Created At,URL,Points,Number of Comments,year,month,day,second,tokenized_title
0,2294635,16043786,top hacker news books in 2017,story,0x54MUR41,2018-01-01 00:06:16,https://hackernewsbooks.com/year/2017,1,0.0,2018,01,01,376,"[top, hacker, news, books, in, 2017]"
1,2294636,16043817,beijing enjoys best winter air quality in five...,story,gpetukhov,2018-01-01 00:15:13,http://www.scmp.com/news/china/policies-politi...,2,0.0,2018,01,01,913,"[beijing, enjoys, best, winter, air, quality, ..."
2,2294637,16043825,ask hn: any domain name registrars that don't ...,ask_hn,glockenspielen,2018-01-01 00:16:54,NaN,1,2.0,2018,01,01,1014,"[ask, hn:, any, domain, name, registrars, that..."
3,2294638,16043845,controversial therapy has led to death threats...,story,cpncrunch,2018-01-01 00:23:05,https://www.buzzfeed.com/tomchivers/inside-the...,2,0.0,2018,01,01,1385,"[controversial, therapy, has, led, to, death, ..."
4,2294639,16043851,ruby 3x3 – ruby 3 will be 3 times faster – wha...,story,geraldbauer,2018-01-01 00:25:05,https://planetruby.github.io/advent2017/ruby3x3,2,0.0,2018,01,01,1505,"[ruby, 3x3, –, ruby, 3, will, be, 3, times, fa..."


In [10]:
def input_phrase(phrase,typ,dict_words):
    for i in phrase:
        if i not in dict_words:
            dict_words[i] = {'story':0,'ask_hn':0,'show_hn':0,'poll':0}
        dict_words[i][typ] += 1
    return dict_words

In [11]:
d = {}
num = 0
for i in range(len(df_2018['tokenized_title'])):
    d = input_phrase(df_2018['tokenized_title'][i],df_2018['Post Type'][i],d)
    num += len(df_2018['tokenized_title'][i])

In [12]:
words_2018 = pd.DataFrame(d)
words_2018 = words_2018.transpose()

In [13]:
words_2018.head()

,ask_hn,poll,show_hn,story
!,1,0,0,1
!!,0,0,0,1
!!`?,0,0,0,1
!=,0,0,0,6
!==,0,0,0,1


In [14]:
delta = 0.5

In [15]:
words_2018['prob_ask_hn'] = words_2018['ask_hn'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_poll'] = words_2018['poll'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_show_hn'] = words_2018['show_hn'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))
words_2018['prob_story'] = words_2018['story'].map(lambda x: (int(x)+delta)/(num+delta*len(words_2018)))

In [16]:
d['poll:']

{'ask_hn': 0, 'poll': 25, 'show_hn': 0, 'story': 11}

In [23]:
words_2018 = words_2018.transpose()

In [24]:
words_2018

,ask_hn,poll,show_hn,story,prob_ask_hn,prob_poll,prob_show_hn,prob_story
!,1.0,0.0,0.0,1.0,6.364408e-07,2.121469e-07,2.121469e-07,6.364408e-07
!!,0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
!!`?,0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
!=,0.0,0.0,0.0,6.0,2.121469e-07,2.121469e-07,2.121469e-07,2.757910e-06
!==,0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
"""+=""",0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
"""12",0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
"""@""",0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
"""[github",0.0,0.0,0.0,1.0,2.121469e-07,2.121469e-07,2.121469e-07,6.364408e-07
"""a11y""",1.0,0.0,0.0,0.0,6.364408e-07,2.121469e-07,2.121469e-07,2.121469e-07


In [21]:
if 'github' in d.keys():
    print(True)

True


In [22]:
d['github']

{'ask_hn': 40, 'poll': 0, 'show_hn': 127, 'story': 1028}